In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [2]:
!git clone https://github.com/google-research/bleurt.git
%cd bleurt
!pip install .

Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116 (from 1)
Receiving objects: 100% (134/134), 31.28 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (49/49), done.
/content/bleurt
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456766 sha256=ece440814a174e6f290ef4783ff2b83d61348ddfc2aa5f9ba6863b9308ff986f
  Stored in directory: /tmp/pip-ephem-wheel-cache-gjb5tcdw/wheels/49/ab/73/9318ab38d4cd1c732bcea8335d3f8d7c0316c8d07b9084fa85
Successfully built BLEURT


In [3]:
!pip install -q evaluate rouge_score bert-score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.4 MB/s eta 0:00:00


In [4]:
import json
import time
import os
import pandas as pd
import numpy as np
import re
from copy import deepcopy

In [5]:
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm

In [ ]:
# !python -m unittest bleurt.score_test
# !python -m unittest bleurt.score_not_eager_test
# !python -m unittest bleurt.finetune_test
# !python -m unittest bleurt.score_files_test

In [6]:
from bleurt import score
from evaluate import load as load_metric

In [7]:
rouge = load_metric("rouge")
bertscore = load_metric("bertscore")
bleurt = load_metric("bleurt", module_type="metric")

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

# Load generations

In [9]:
fine_tune_path = '/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM'
model_name = 'omi_health' # ADJUST
sub_model_name = 'pegasus_pubmed' # ADJUST
checkpoints_dir = f"{fine_tune_path}/{model_name}/{sub_model_name}/lora_1" # ADJUST
checkpoints_path = f"{checkpoints_dir}/checkpoints"
final_checkpoints_path = f"{checkpoints_dir}/final_checkpoints"
eval_path = f"{checkpoints_dir}/eval_{sub_model_name}.xlsx"

In [10]:
eval_path = f"{fine_tune_path}/{model_name}"

In [11]:
eval_path

'/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/omi_health'

In [12]:
checkpoints_dir

'/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/omi_health/pegasus_pubmed/lora_1'

In [13]:
test_soap_15_df_tmp = pd.read_excel(f"{eval_path}/omi_health_test_15.xlsx")

In [14]:
test_soap_15_df_tmp.head(3)

,input,output,instruction,generated_summary,inference_time_sec,input_token_length,generated_token_length,inference_error
0,"Doctor: Hello, can you please tell me about yo...","S: The patient, a flooring installer with no s...",NaN,S: The patient reports experiencing painless b...,314.399537,759,508,NaN
1,"Doctor: Hello, I understand that you're a 7-ye...",S: The patient is a 7-year-old boy with congen...,NaN,S: The patient is a 7-year-old boy with congen...,314.399537,759,508,NaN
2,"Doctor: Hello, we've received your results fro...",S: The patient reported undergoing an ultrasou...,NaN,S: The patient reports experiencing painless b...,314.399537,759,508,NaN


In [15]:
test_soap_50_df = test_soap_15_df_tmp[['output', 'generated_summary']].rename(columns={'output': 'reference_note', 'generated_summary': 'fine_tuned_model_output'})

In [16]:
print(len(test_soap_50_df))
test_soap_50_df.head(3)

15


,reference_note,fine_tuned_model_output
0,"S: The patient, a flooring installer with no s...",S: The patient reports experiencing painless b...
1,S: The patient is a 7-year-old boy with congen...,S: The patient is a 7-year-old boy with congen...
2,S: The patient reported undergoing an ultrasou...,S: The patient reports experiencing painless b...


# Evaluate on MTS-Dialog

## Parallel

In [17]:
# Parallel
def evaluate_in_batch(df, batch_size=16):
    results = []

    for start in tqdm(range(0, len(df), batch_size), desc="Evaluating in batches"):
        batch = df.iloc[start:start+batch_size]
        preds = batch["fine_tuned_model_output"].tolist()
        refs = batch["reference_note"].tolist()
        diags = batch["dialogue"].tolist()

        # Compute metrics
        # rouge_scores = rouge.compute(predictions=preds, references=refs, use_stemmer=True)
        bert_scores = bertscore.compute(predictions=preds, references=refs, lang="en", device="cuda")
        bleurt_scores = bleurt.compute(predictions=preds, references=refs)

        for i in range(len(preds)):
          # ROUGE must be computed per example
            rouge_score = rouge.compute(predictions=[preds[i]], references=[refs[i]], use_stemmer=True)

            results.append({
                "dialogue": diags[i],
                "reference_summary": refs[i],
                "generated_summary": preds[i],
                "ROUGE-1": round(rouge_score['rouge1'], 4),
                "ROUGE-2": round(rouge_score['rouge2'], 4),
                "ROUGE-L": round(rouge_score['rougeL'], 4),
                "ROUGE-LSum": round(rouge_score['rougeLsum'], 4),
                "BERTScore-P": round(bert_scores['precision'][i], 4),
                "BERTScore-R": round(bert_scores['recall'][i], 4),
                "BERTScore-F1": round(bert_scores['f1'][i], 4),
                "BLEURT": round(bleurt_scores['scores'][i], 4),
            })

    return pd.DataFrame(results)

In [18]:
f"{eval_path}/omi_health_test_15.xlsx"

'/content/drive/MyDrive/ClinicalNotesGen/Summarization/3_Fine_Tune_LLM/omi_health/omi_health_test_15.xlsx'

In [20]:
test_soap_50_df["dialogue"] = pd.NA
eval_test_soap_50_df = evaluate_in_batch(test_soap_50_df, batch_size=16)
test_soap_15_df_tmp = pd.read_excel(f"{eval_path}/omi_health_test_15.xlsx")
# eval_test_soap_50_df.to_csv(f"{eval_path}/eval_test_soap_50.csv", index=False)

Evaluating in batches:   0%|          | 0/1 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Evaluating in batches: 100%|██████████| 1/1 [00:25<00:00, 25.39s/it]


In [21]:
eval_test_soap_50_df

,dialogue,reference_summary,generated_summary,ROUGE-1,ROUGE-2,ROUGE-L,ROUGE-LSum,BERTScore-P,BERTScore-R,BERTScore-F1,BLEURT
0,<NA>,"S: The patient, a flooring installer with no s...",S: The patient reports experiencing painless b...,0.7120,0.4773,0.5372,0.6084,0.9177,0.9045,0.9111,0.1975
1,<NA>,S: The patient is a 7-year-old boy with congen...,S: The patient is a 7-year-old boy with congen...,0.8729,0.7611,0.8398,0.8508,0.9450,0.9660,0.9554,0.3743
2,<NA>,S: The patient reported undergoing an ultrasou...,S: The patient reports experiencing painless b...,0.3077,0.0539,0.1646,0.2540,0.8235,0.8278,0.8256,-0.5931
3,<NA>,"S: The patient reports a progressive headache,...","S: The patient reports progressive headache, n...",0.6609,0.3799,0.4826,0.5783,0.9231,0.9158,0.9194,0.5039
4,<NA>,"S: The patient, a post-liver transplant recipi...","S: The patient, with a history of hepatocellul...",0.5866,0.2943,0.3463,0.5053,0.9031,0.9064,0.9048,0.0482
5,<NA>,"S: The patient, a non-smoker and engineer, rep...","S: The patient, a non-smoker and engineer, pre...",0.6872,0.5000,0.5507,0.6344,0.9473,0.9286,0.9379,0.1520
6,<NA>,S: The patient presented with body weight loss...,"S: The patient, with a history of well-control...",0.7257,0.5254,0.5359,0.6624,0.9239,0.9330,0.9284,-0.0612
7,<NA>,S: The patient reports a 6-month history of sw...,S: The patient reports a 6-month history of sw...,0.8103,0.6147,0.6940,0.7284,0.9574,0.9466,0.9519,0.3425
8,<NA>,"S: The patient, a male with a history of prost...","S: The patient, a male with a history of prost...",0.7534,0.5052,0.6164,0.6678,0.9377,0.9356,0.9366,0.3969
9,<NA>,S: The patient is a 76-year-old woman presenti...,S: The patient is a 76-year-old woman presenti...,0.7605,0.5431,0.6399,0.6764,0.9419,0.9297,0.9358,0.3686


In [22]:
def calculate_average_metrics(df):
    metrics = [
        'ROUGE-1', 'ROUGE-2', 'ROUGE-L', 'ROUGE-LSum',
        'BERTScore-P', 'BERTScore-R', 'BERTScore-F1', 'BLEURT'
    ]
    avg_scores = {metric: df[metric].mean() for metric in metrics}
    return avg_scores

In [23]:
def display_evaluation(average_scores):
    print("Average Evaluation Metrics:")
    for metric, score in average_scores.items():
        print(f"{metric}: {score:.4f}")

In [24]:
eval_test_soap = calculate_average_metrics(eval_test_soap_50_df) # modified soap summary

In [25]:
display_evaluation(eval_test_soap)

Average Evaluation Metrics:
ROUGE-1: 0.7011
ROUGE-2: 0.4733
ROUGE-L: 0.5529
ROUGE-LSum: 0.6318
BERTScore-P: 0.9251
BERTScore-R: 0.9226
BERTScore-F1: 0.9238
BLEURT: 0.2081
